In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op

from mod import load_data, alt_theme

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [4]:
# Parameters
renderer = "kaggle"
start_date = "2019-05-01"
end_date = "2019-06-01"


In [5]:
comments, issues, prs = load_data('../data/')
bot_names = pd.read_csv('bot_names.csv')['names'].tolist()
comments = comments.query('author not in @bot_names')
issues = issues.query('author not in @bot_names')
prs = prs.query('author not in @bot_names')

/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  comments = pd.concat(comments)


/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  issues = pd.concat(issues)
/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  prs = pd.concat(prs)


In [6]:
# Only keep the dates we want
comments = comments.query('updatedAt > @start_date and updatedAt < @end_date')
issues = issues.query('updatedAt > @start_date and updatedAt < @end_date')
prs = prs.query('updatedAt > @start_date and updatedAt < @end_date')

In [7]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [8]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [9]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [10]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/nteract](https://github.com/nteract)

In [11]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-05-01** to **2019-06-01**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [12]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 31 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [13]:
merged = prs.query('state == "MERGED" and closedAt > @start_date and closedAt < @end_date')

In [14]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                       repo  author
  54          nteract                    nteract      12
  53          nteract                   hydrogen       7
  55          nteract                  papermill       7
  56          nteract                  scrapbook       3
  57          nteract                   semiotic       2
  52          nteract                   commuter       1
  58          nteract                       vdom       1
  47       jupyterlab                 jupyterlab     265
  51       jupyterlab          jupyterlab_server      25
  46       jupyterlab          jupyter-renderers      20
  48       jupyterlab             jupyterlab-git      10
  45       jupyterlab  extension-cookiecutter-js       5
  49       jupyterlab             jupyterlab-toc       5
  50       jupyterlab        jupyterlab_pygments       5
  42       jupyterhub    the-littlest-jupyterhub      48
  38       jupyterhub        mybinder.org-deploy      38
  44       jupyterhub     zero-to-jupyterhub-k8s      28
  34       jupyterhub                 jupyterhub      22
  30       jupyterhub                  binderhub      12
  39       jupyterhub                nbgitpuller       8
  28       jupyterhub       alabaster-jupyterhub       6
  31       jupyterhub                 chartpress       4
  33       jupyterhub       jupyter-server-proxy       4
  40       jupyterhub             oauthenticator       4
  41       jupyterhub               team-compass       4
  29       jupyterhub                     binder       2
  32       jupyterhub     jupyter-rsession-proxy       2
  35       jupyterhub             jupyterlab-hub       2
  36       jupyterhub                kubespawner       2
  37       jupyterhub           ltiauthenticator       2
  43       jupyterhub           tmpauthenticator       2
  25  jupyter-widgets                 ipyleaflet      52
  26  jupyter-widgets                 ipywidgets      16
  27  jupyter-widgets                  pythreejs       8
  20          jupyter                   nbgrader      19
  24          jupyter                repo2docker      18
  18          jupyter                  nbconvert      16
  9           jupyter              docker-stacks      11
  22          jupyter                   notebook       8
  23          jupyter                  qtconsole       8
  10          jupyter         enterprise_gateway       7
  12          jupyter               jupyter-book       6
  13          jupyter             jupyter-sphinx       5
  14          jupyter             jupyter_client       4
  16          jupyter             kernel_gateway       3
  11          jupyter                    jupyter       2
  15          jupyter             jupyter_server       2
  17          jupyter                      nb2kg       1
  19          jupyter                     nbdime       1
  21          jupyter                   nbviewer       1
  5           ipython                    ipython      24
  4           ipython                ipyparallel       6
  3           ipython                  ipykernel       3
  6           ipython           ipython-in-depth       3
  7           ipython            ipython-website       3
  8           ipython                  traitlets       3
  0   binder-examples         getting-data-xroot       2
  1   binder-examples                      latex       2
  2   binder-examples           python-conda_pip       2,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['nteract', 'hydrogen', 'papermill', 'scrapbook', 'semiotic', 'commuter', 'vdom', 'jupyterlab', 'jupyterlab_server', 'jupyter-renderers', 'jupyterlab-git', 'extension-cookiecutter-js', 'jupyterlab-toc', 'jupyterlab_pygments', 'the-littlest-jupyterhub', 'mybinder.org-deploy', 'zero-to-jupyterhub-k8s', 'jupyterhub', 'binderhub', 'nbgitpuller', 'alabaster-jupyterhub', 'chartpress', 'jupyter-server-proxy', 'oauthenticator'

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [15]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [16]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [17]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:           username  authored authorAssociation
    0        fomightez         2       CONTRIBUTOR
    1  matthewfeickert         2       CONTRIBUTOR
    2        yuvipanda         2            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:          username  authored authorAssociation
    7           minrk         9            MEMBER
    0       Pragnya02         3       CONTRIBUTOR
    1            alok         3       CONTRIBUTOR
    2        bashtage         3       CONTRIBUTOR
    3   frenzymadness         3       CONTRIBUTOR
    4        gpotter2         3       CONTRIBUTOR
    5           k-zen         3       CONTRIBUTOR
    6       mbbroberg         3       CONTRIBUTOR
    8           rgson         3       CONTRIBUTOR
    9        stonebig         3       CONTRIBUTOR
    10      tacaswell         3       CONTRIBUTOR
    11        zzzz-qq         3       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for ipython in the last 31 days',
    width: 1000
  }), Chart({
    data:           username  authored       authorAssociation
    16         betatim         9                  MEMBER
    35        jhamrick         9                  MEMBER
    18       choldgraf         7                  MEMBER
    48         parente         6                  MEMBER
    17      ccordoba12         5            COLLABORATOR
    54        t-makaro         5            COLLABORATOR
    34        jbweston         4                  MEMBER
    7    SylvainCorlay         3                  MEMBER
    11       akchinSTC         3            COLLABORATOR
    56        willingc         3                  MEMBER
    13        alexrudy         3             CONTRIBUTOR
    50         rkdarst         3             CONTRIBUTOR
    3        LuisARodr         2             CONTRIBUTOR
    24           dleen         2             CONTRIBUTOR
    20        dalthviz         2             CONTRIBUTOR
    44           minrk         2                  MEMBER
    10         Zsailer         2                  MEMBER
    9       Xarthisius         2             CONTRIBUTOR
    43       michzimny         1             CONTRIBUTOR
    39       krishnatg         1                    NONE
    37      jrbourbeau         1             CONTRIBUTOR
    36      joergbrech         1             CONTRIBUTOR
    40      manycoding         1             CONTRIBUTOR
    41      marcraminv         1             CONTRIBUTOR
    42     martinRenou         1             CONTRIBUTOR
    38     kevin-bates         1                  MEMBER


In [18]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                org                repo   username  closed
    0  binder-examples  getting-data-xroot  choldgraf       2
    1  binder-examples               latex  choldgraf       2
    2  binder-examples    python-conda_pip  choldgraf       2,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:        org              repo   username  closed
    3  ipython         ipykernel  blink1073       3
    4  ipython       ipyparallel      minrk       6
    5  ipython           ipython    Carreau      24
    6  ipython  ipython-in-depth  blink1073       3
    7  ipython   ipython-website  blink1073       3
    8  ipython         traitlets   rmorshea       3,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 31 days',
    width: 1000
  }), Chart({
    data:         org                repo       username  closed
    9   jupyter       docker-stacks        parente      11
    10  jupyter  enterprise_gateway    kevin-bates       5
    11  jupyter  enterprise_gateway       lresende       2
    12  jupyter             jupyter      choldgraf       1
    13  jupyter             jupyter        parente       1
    14  jupyter        jupyter-book      choldgraf       6
    15  jupyter      jupyter-sphinx       akhmerov       3
    16  jupyter      jupyter-sphinx       jbweston       2
    17  jupyter      jupyter_client          MSeal       1
    18  jupyter      jupyter_client      blink1073       1
    19  jupyter      jupyter_client          minrk       1
    20  jupyter      jupyter_client         mpacer       1
    21  jupyter      jupyter_server  SylvainCorlay       1
    22  jupyter      jupyter_server         ivanov       1
    23  jupyter      kernel_gateway       rolweber       3
    24  jupyter               nb2kg    kevin-bates       1
    25  jupyter           nbconvert          MSeal      13
    26  jupyter           nbconvert  SylvainCorlay       1
    27  jupyter           nbconvert       t-makaro       2
    28  jupyter              nbdime        vidartf       1
    29  jupyter            nbgrader       jhamrick      19
    30  jupyter            nbviewer        parente       1
    31  jupyter            notebook          minrk       2
    32  jupyter            notebook      takluyver       6
    33  jupyter           qtconsole     ccordoba12       8
    34  jupyter         repo2docker        betatim       9
    35  jupyter         repo2docker      choldgraf       2
    36  jupyter         repo2docker   consideRatio       1
    37  jupyter         repo2docker          minrk       1
    38  jupyter         repo2docker      yuvipanda       5,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter in the last 31 days',
    width: 1000
  }), Chart({
    data:                 org        repo     username  closed
    39  jupyter-widgets  ipyleaflet  martinRenou      52
    40  jupyter-widgets  ipywidgets   jasongrout      16
    41  jupyter-widgets   pythreejs      vidartf       8,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter-widgets in the last 31 days',
    width: 1000
  }), Chart({
    data:            org                     repo        username  closed
    42  jupyterhub

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                      repo  number  \
  0           ipython                 ipykernel      12   
  1           ipython               ipyparallel       9   
  2           ipython                   ipython      60   
  3           jupyter                dashboards       1   
  4           jupyter             docker-stacks       1   
  5           jupyter        enterprise_gateway       3   
  6           jupyter                   jupyter       1   
  7           jupyter              jupyter-book       4   
  8           jupyter            jupyter-sphinx       1   
  9           jupyter            jupyter_client       1   
  10          jupyter            jupyter_server       2   
  11          jupyter            kernel_gateway       1   
  12          jupyter                 nbconvert       2   
  13          jupyter                    nbdime       2   
  14          jupyter                  nbgrader       4   
  15          jupyter                  notebook      27   
  16          jupyter                 qtconsole       1   
  17          jupyter               repo2docker       8   
  18          jupyter                  steering       1   
  19          jupyter                 terminado       1   
  20  jupyter-widgets                ipyleaflet      24   
  21  jupyter-widgets                ipywidgets      44   
  22  jupyter-widgets                 pythreejs       8   
  23       jupyterhub      alabaster-jupyterhub       2   
  24       jupyterhub                 binderhub      23   
  25       jupyterhub             dockerspawner      10   
  26       jupyterhub        dummyauthenticator       4   
  27       jupyterhub    jupyter-rsession-proxy       2   
  28       jupyterhub      jupyter-server-proxy       4   
  29       jupyterhub                jupyterhub       4   
  30       jupyterhub  jupyterhub-deploy-docker       3   
  31       jupyterhub            jupyterlab-hub       8   
  32       jupyterhub               kubespawner       4   
  33       jupyterhub         ldapauthenticator       2   
  34       jupyterhub          ltiauthenticator       2   
  35       jupyterhub       mybinder.org-deploy       6   
  36       jupyterhub       nativeauthenticator       1   
  37       jupyterhub               nbgitpuller       6   
  38       jupyterhub            oauthenticator      10   
  39       jupyterhub               sudospawner       2   
  40       jupyterhub            systemdspawner       2   
  41       jupyterhub              team-compass       6   
  42       jupyterhub   the-littlest-jupyterhub      14   
  43       jupyterhub    zero-to-jupyterhub-k8s      26   
  44       jupyterlab         jupyter-renderers       5   
  45       jupyterlab                jupyterlab     139   
  46       jupyterlab       jupyterlab-celltags      15   
  47       jupyterlab            jupyterlab-git      14   
  48       jupyterlab            jupyterlab-toc       5   
  49          nteract                 bookstore       5   
  50          nteract                  commuter       1   
  51          nteract        commuter-on-glitch       1   
  52          nteract                   nteract       7   
  53          nteract                nteract.io       2   
  54          nteract                 papermill       8   
  55          nteract                 scrapbook       1   
  56          nteract                  semiotic       4   
  
                                 org/repo  
  0                     ipython/ipykernel  
  1                   ipython/ipyparallel  
  2                       ipython/ipython  
  3                    jupyter/dashboards  
  4                 jupyter/docker-stacks  
  5            jupyter/enterprise_gateway  
  6                       jupyter/jupyter  
  7                  jupyter/jupyter-book  
  8                jupyter/jupyter-sphinx  
  9                jupyter/jupyter_client  
  10               jupyter/jupyter_server  
  11               jupyter/kernel

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                       repo  number  \
  0           ipython                  ipykernel       6   
  1           ipython                ipyparallel       3   
  2           ipython                    ipython      45   
  3           jupyter              atom-notebook       1   
  4           jupyter              docker-stacks       3   
  5           jupyter         enterprise_gateway      13   
  6           jupyter                    jupyter       2   
  7           jupyter               jupyter-book      10   
  8           jupyter             jupyter-sphinx       7   
  9           jupyter             jupyter_client       1   
  10          jupyter               jupyter_core       1   
  11          jupyter             jupyter_server       6   
  12          jupyter             kernel_gateway       3   
  13          jupyter                      nb2kg       1   
  14          jupyter                  nbconvert      10   
  15          jupyter                   nbgrader      21   
  16          jupyter                   nbviewer       3   
  17          jupyter                   notebook       9   
  18          jupyter                  qtconsole       5   
  19          jupyter                repo2docker      13   
  20          jupyter                  terminado       1   
  21  jupyter-widgets                 ipyleaflet      28   
  22  jupyter-widgets                 ipywidgets      36   
  23  jupyter-widgets                  pythreejs       8   
  24       jupyterhub       alabaster-jupyterhub       2   
  25       jupyterhub               batchspawner       2   
  26       jupyterhub                  binderhub      10   
  27       jupyterhub    configurable-http-proxy       2   
  28       jupyterhub       jupyter-server-proxy       8   
  29       jupyterhub                 jupyterhub      24   
  30       jupyterhub                kubespawner       4   
  31       jupyterhub          ldapauthenticator       2   
  32       jupyterhub        mybinder.org-deploy       8   
  33       jupyterhub                nbgitpuller      18   
  34       jupyterhub             oauthenticator       1   
  35       jupyterhub             systemdspawner       2   
  36       jupyterhub               team-compass       8   
  37       jupyterhub    the-littlest-jupyterhub      72   
  38       jupyterhub           tmpauthenticator       2   
  39       jupyterhub     zero-to-jupyterhub-k8s       8   
  40       jupyterlab  extension-cookiecutter-js       5   
  41       jupyterlab          jupyter-renderers       5   
  42       jupyterlab                 jupyterlab     302   
  43       jupyterlab   jupyterlab-data-explorer       5   
  44       jupyterlab             jupyterlab-git      15   
  45       jupyterlab             jupyterlab-toc       5   
  46          nteract                  bookstore       4   
  47          nteract                   commuter       1   
  48          nteract         commuter-on-glitch       1   
  49          nteract                   hydrogen       3   
  50          nteract                    nteract       5   
  51          nteract                  papermill       6   
  52          nteract                  scrapbook       1   
  53          nteract                   semiotic       9   
  
                                  org/repo  
  0                      ipython/ipykernel  
  1                    ipython/ipyparallel  
  2                        ipython/ipython  
  3                  jupyter/atom-notebook  
  4                  jupyter/docker-stacks  
  5             jupyter/enterprise_gateway  
  6                        jupyter/jupyter  
  7                   jupyter/jupyter-book  
  8                 jupyter/jupyter-sphinx  
  9                 jupyter/jupyter_client  
  10                  jupyter/jupyter_core  
  11                jupyter/jupyter_server  
  12                jupyter/kernel_gateway  
  13                         jupyter/nb2kg  
  14    

In [22]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [23]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/altair/utils/core.py:90: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  "Defaulting to nominal.".format(typ))


HConcatChart({
  hconcat: [Chart({
    data:             org                       repo     kind  count
    44   jupyterlab          jupyter-renderers  created    5.0
    45   jupyterlab                 jupyterlab  created  139.0
    46   jupyterlab        jupyterlab-celltags  created   15.0
    47   jupyterlab             jupyterlab-git  created   14.0
    48   jupyterlab             jupyterlab-toc  created    5.0
    64   jupyterlab  extension-cookiecutter-js  created    0.0
    65   jupyterlab   jupyterlab-data-explorer  created    0.0
    111  jupyterlab          jupyter-renderers   closed    5.0
    112  jupyterlab                 jupyterlab   closed  302.0
    113  jupyterlab        jupyterlab-celltags   closed    0.0
    114  jupyterlab             jupyterlab-git   closed   15.0
    115  jupyterlab             jupyterlab-toc   closed    5.0
    131  jupyterlab  extension-cookiecutter-js   closed    5.0
    132  jupyterlab   jupyterlab-data-explorer   closed    5.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for jupyterlab',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:                 org        repo     kind  count
    20  jupyter-widgets  ipyleaflet  created   24.0
    21  jupyter-widgets  ipywidgets  created   44.0
    22  jupyter-widgets   pythreejs  created    8.0
    87  jupyter-widgets  ipyleaflet   closed   28.0
    88  jupyter-widgets  ipywidgets   closed   36.0
    89  jupyter-widgets   pythreejs   closed    8.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for jupyter-widgets',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:         org         repo     kind  count
    0   ipython    ipykernel  created   12.0
    1   ipython  ipyparallel  created    9.0
    2   ipython      ipython  created   60.0
    67  ipython    ipykernel   closed    6.0
    68  ipython  ipyparallel   closed    3.0
    69  ipython      ipython   closed   45.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for ipython',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data: Empty DataFrame
    Columns: [org, repo, kind, count]
    Index: [],
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
      

In [24]:
# Set to datetime
for kind in ['createdAt', 'closedAt']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closedAt'] - closed['createdAt']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [25]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
    color='org',
)

Chart({
  data:                 org                       repo   time_open  \
  0           ipython                  ipykernel  159.198507   
  1           ipython                ipyparallel   62.380706   
  2           ipython                    ipython   12.944410   
  3           jupyter              atom-notebook   73.856030   
  4           jupyter              docker-stacks    7.612419   
  5           jupyter         enterprise_gateway    7.042882   
  6           jupyter                    jupyter  196.567251   
  7           jupyter               jupyter-book   23.872500   
  8           jupyter             jupyter-sphinx    8.418785   
  9           jupyter             jupyter_client  352.613438   
  10          jupyter               jupyter_core  540.746690   
  11          jupyter             jupyter_server  171.686580   
  12          jupyter             kernel_gateway  346.509884   
  13          jupyter                      nb2kg   12.882593   
  14          jupyter                  nbconvert   53.619676   
  15          jupyter                   nbgrader  213.927789   
  16          jupyter                   nbviewer    0.436933   
  17          jupyter                   notebook    6.022847   
  18          jupyter                  qtconsole   67.087928   
  19          jupyter                repo2docker  225.174444   
  20          jupyter                  terminado    0.076042   
  21  jupyter-widgets                 ipyleaflet   13.138449   
  22  jupyter-widgets                 ipywidgets    1.311250   
  23  jupyter-widgets                  pythreejs    6.722697   
  24       jupyterhub       alabaster-jupyterhub  223.006389   
  25       jupyterhub               batchspawner  719.652072   
  26       jupyterhub                  binderhub    1.050660   
  27       jupyterhub    configurable-http-proxy  504.969387   
  28       jupyterhub       jupyter-server-proxy    0.061719   
  29       jupyterhub                 jupyterhub    5.379641   
  30       jupyterhub                kubespawner    0.284722   
  31       jupyterhub          ldapauthenticator    0.985625   
  32       jupyterhub        mybinder.org-deploy  116.222951   
  33       jupyterhub                nbgitpuller  366.100509   
  34       jupyterhub             oauthenticator   44.931007   
  35       jupyterhub             systemdspawner   20.067164   
  36       jupyterhub               team-compass    4.705938   
  37       jupyterhub    the-littlest-jupyterhub  116.817674   
  38       jupyterhub           tmpauthenticator   50.088738   
  39       jupyterhub     zero-to-jupyterhub-k8s    2.279873   
  40       jupyterlab  extension-cookiecutter-js    0.951539   
  41       jupyterlab          jupyter-renderers   11.933252   
  42       jupyterlab                 jupyterlab   14.334954   
  43       jupyterlab   jupyterlab-data-explorer  144.505486   
  44       jupyterlab             jupyterlab-git    1.032477   
  45       jupyterlab             jupyterlab-toc   90.226829   
  46          nteract                  bookstore    1.237512   
  47          nteract                   commuter   13.431551   
  48          nteract         commuter-on-glitch  188.914167   
  49          nteract                   hydrogen    2.505451   
  50          nteract                    nteract    1.614549   
  51          nteract                  papermill   16.825226   
  52          nteract                  scrapbook    0.461551   
  53          nteract                   semiotic   21.760567   
  
                                  org/repo  
  0                      ipython/ipykernel  
  1                    ipython/ipyparallel  
  2                        ipython/ipython  
  3                  jupyter/atom-notebook  
  4                  jupyter/docker-stacks  
  5             jupyter/enterprise_gateway  
  6                        jupyter/jupyter  
  7                   jupyter/jupyter-book  
  8                 jupyter/jupyter-sphinx  
  9          

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [26]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'issue_id'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [27]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repo, i_issues in idata.groupby('repo'):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")

        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 31 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [28]:
commentors = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                 author  count authorAssociation
    7             stwunsch     16              NONE
    2              betatim      8            MEMBER
    6      matthewfeickert      6              NONE
    1  3keepmovingforward3      4              NONE
    4            choldgraf      4            MEMBER
    8         taylorreiter      4              NONE
    0               0lorak      2              NONE
    3             cboettig      2            MEMBER
    5            chrisburr      2              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:                     author  count authorAssociation
    3                  Carreau    156            MEMBER
    63                rmorshea     33      COLLABORATOR
    46             kevin-bates     21       CONTRIBUTOR
    41                 hroncok     18       CONTRIBUTOR
    69                stonebig     18       CONTRIBUTOR
    38                gpotter2     15       CONTRIBUTOR
    52                   minrk     15            MEMBER
    33                 efiring     12       CONTRIBUTOR
    18                asmeurer      9       CONTRIBUTOR
    66            sanketShetty      9              NONE
    21               blink1073      9            MEMBER
    32             drunkwcodes      9              NONE
    54                  natsrm      9              NONE
    27                 dashesy      6              NONE
    60              pierstitus      6       CONTRIBUTOR
    35            flying-sheep      6       CONTRIBUTOR
    24                   cdeil      6              NONE
    62                   rgson      6       CONTRIBUTOR
    53                 mlucool      6              NONE
    29                dhomeier      6              NONE
    45        jonathanslenders      6       CONTRIBUTOR
    15          angertdevsingh      6              NONE
    70               tacaswell      6       CONTRIBUTOR
    4           LucianaMarques      6       CONTRIBUTOR
    9                Pragnya02      6       CONTRIBUTOR
    17            arogozhnikov      6              NONE
    64                rudaoshi      3              NONE
    73           thequackdaddy      3              NONE
    47                   kmtac      3              NONE
    48                  lheagy      3              NONE
    49             martinRenou      3              NONE
    75                   zewpo      3              NONE
    50                memeplex      3       CONTRIBUTOR
    51            michaelort33      3              NONE
    74                  yluthu      3              NONE
    71               takluyver      3            MEMBER
    72              texadactyl      3              NONE
    55                 njsmith      3       CONTRIBUTOR
    56                  noklam      3              NONE
    57             petebachant      3              NONE
    58                 pfortho      3              NONE
    68                 scopatz      3       CONTRIBUTOR
    59                pganssle      3       CONTRIBUTOR
    67                 schoreZ      3              NONE
    65               rueberger      3              NONE
    61             psychemedia      3              NONE
    0   AbsurdMagp

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [30]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

In [31]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [32]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                 author  n_first_responses authorAssociation
    0  3keepmovingforward3                  1              NONE
    1            choldgraf                  1            MEMBER
    2      matthewfeickert                  1       CONTRIBUTOR
    3             stwunsch                  1              NONE
    4         taylorreiter                  1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:                     author  n_first_responses authorAssociation
    2                  Carreau                 26            MEMBER
    37                   minrk                  5            MEMBER
    45                rmorshea                  3      COLLABORATOR
    22             drunkwcodes                  3              NONE
    28                gpotter2                  3       CONTRIBUTOR
    16                   cdeil                  2              NONE
    31                 hroncok                  2       CONTRIBUTOR
    48                stonebig                  2       CONTRIBUTOR
    18                 dashesy                  2              NONE
    0   AbsurdMagpieScrutinies                  1              NONE
    30            hrishikeshrt                  1              NONE
    32              ikizhvatov                  1              NONE
    33              jamesmyatt                  1              NONE
    34                  lheagy                  1              NONE
    35                memeplex                  1       CONTRIBUTOR
    36            michaelort33                  1              NONE
    39                  noklam                  1              NONE
    38                  natsrm                  1              NONE
    40             petebachant                  1              NONE
    41                 pfortho                  1              NONE
    42              pierstitus                  1       CONTRIBUTOR
    43             psychemedia                  1              NONE
    44                   rgson                  1       CONTRIBUTOR
    46                rudaoshi                  1              NONE
    47            sanketShetty                  1              NONE
    49               tacaswell                  1       CONTRIBUTOR
    50           thequackdaddy                  1              NONE
    29              hmswaffles                  1              NONE
    26           frenzymadness                  1       CONTRIBUTOR
    27               g3ntelm3n                  1              NONE
    12                asmeurer                  1       CONTRIBUTOR
    3           LucianaMarques                  1       CONTRIBUTOR
    4                   MInner                  1              NONE
    5                    MSeal                  1       CONTRIBUTOR
    6                  Madhu94                  1       CONTRIBUTOR
    7          NotWearingPants                  1              NONE
    8          Shahinur-Shakib                  1              NONE
    9          StanczakDominik                  1              NONE
    10        aggregate1166877                  1              NONE
    11                 andersk                  

In [33]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>